In [1]:
%load_ext autoreload
%autoreload 2

# Combining Results in ORD

This notebook provides a proof of concept example for combining catastrophe
loss model results in the Open Results Data (ORD) format. We follow the
methodology outlined in *Combining_results_in_ORD_v1.1.pdf*.

This notebook is split into the workflow sequence as follows:

1. Load and Group
2. Period Sampling
3. Loss Sampling
4. Output Preparation

In [2]:
# imports
from datetime import datetime
from pathlib import Path
import json
from dataclasses import asdict
import pandas as pd

In [3]:
# make sure relative imports work
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The input files are multiple runs of PiWind.

In [4]:
parent_path = Path('~/code/ODS_Tools/ord_combining').expanduser()

ord_output_dirs = [parent_path / "ord-losses-1/output",
                   parent_path / "ord-losses-2/output",
                   parent_path / "ord-losses-3/output"]

In [5]:
# specify directory for outputs

output_dir = Path("./combined_ord-" + datetime.now().strftime("%d%m%y%H%M%S"))
output_dir.mkdir(exist_ok=True)
print(f'Output Path: {output_dir}')

Output Path: combined_ord-191125150622


## 1. Load and Group
### Creating Analysis and OutputSet
In this section we create the objects required prior to grouping, namely:
- Analysis table which contains the meta data from the analyses
- OutputSet table which contains references to the ORD results.


The `analysis_settings.json` files for each ORD analysis are parsed to read the Analysis and OutputSet tables.

In [6]:
from ord_combining.outputset import parse_analysis_settings
from ord_combining.common import dataclass_list_to_dataframe

def load_analysis_and_output_sets(ord_output_dirs):
    analysis_set = []
    output_sets = []
    analysis_id = 1
    for ord_dir in ord_output_dirs:
        analysis, _outputsets = parse_analysis_settings(ord_dir / 'analysis_settings.json')

        # set the uid for the analysis
        analysis.id = analysis_id
        for i in range(len(_outputsets)):
            _outputsets[i].analysis_id = analysis_id
        analysis_id += 1

        analysis_set.append(analysis)
        output_sets += _outputsets

    return analysis_set, output_sets

analysis, outputsets = load_analysis_and_output_sets(ord_output_dirs)
analysis = {a.id: a for a in analysis}
outputsets_df = dataclass_list_to_dataframe(outputsets)
outputsets_df['id'] = outputsets_df.index # set id col

In [7]:
outputsets_df

,id,perspective_code,analysis_id,exposure_summary_level_fields,exposure_summary_level_id
0,0,gul,1,[],1
1,1,gul,1,[LocNumber],2
2,2,gul,1,[PolNumber],3
3,3,il,1,[],1
4,4,il,1,[LocNumber],2
5,5,gul,2,[],1
6,6,gul,2,[LocNumber],2
7,7,gul,2,[OccupancyCode],3
8,8,gul,3,[],1
9,9,gul,3,[LocNumber],2


In [8]:
outputsets_df.columns

Index(['id', 'perspective_code', 'analysis_id',
       'exposure_summary_level_fields', 'exposure_summary_level_id'],
      dtype='object')

### Creating GroupEventSet
The GroupEventSet are used to define common events, thereby allowing for a
list of consistent unique events that can be used to create GroupPeriods.

There is a config option `group_event_set_fields` which specifies which fields to use to specify the unique event.

The EventOccurenceSet table contains the meta information for each event set based on the `group_event_set_fields`.

In [9]:
from ord_combining.groupeventset import generate_group_set, generate_group_event_set
group_event_set_fields = ['event_set_id', 'event_occurrence_id', 'model_supplier_id']

group_set, group_output_set = generate_group_set(outputsets_df)
event_occurrence_set_df, group_event_set_analysis = generate_group_event_set(analysis, group_event_set_fields)

In [10]:
group_output_set

{0: 0, 1: 1, 2: 3, 3: 4, 4: 5, 5: 0, 6: 1, 7: 2, 8: 0, 9: 1}

In [11]:
group_set

,group_id,perspective_code,exposure_summary_level_fields_string
group_set_id,,,
0,1,gul,
1,1,gul,LocNumber
2,1,gul,OccupancyCode
3,1,gul,PolNumber
4,1,il,
5,1,il,LocNumber


In [12]:
event_occurrence_set_df

,id,event_set_id,event_occurrence_id,model_supplier_id
0,1,p,lt,OasisLMF
1,2,p,st,OasisLMF


In [13]:
group_event_set_analysis

,analysis_id,group_event_set_id
0,1,1
1,2,1
2,3,2


Once the groups have been assigned the SummaryId is aligned within each group_set.
To do so we find each unique grouping of summary level fields in each group set and aggregate the tiv by summing.
Then we produce a `outputset_summary_id_map` which contains dicts which maps
the summary_id of the ORD files to the group `SummaryId` indexed by a key
value of `output_set_id`.
Note only adds mapping where summary_id != SummaryId

To demo this swapped LocNumber for summary_id 1 and 2 in /home/vinulw/code/ODS_Tools/ord_combining/losses-20251021131718 SummaryLevel 2

In [14]:
from ord_combining.summaryinfo import load_summary_info, assign_summary_ids, generate_summary_id_map
os_summary_info = load_summary_info(analysis, outputsets_df)
group_set_summary_info = assign_summary_ids(group_output_set, os_summary_info)

In [15]:

outputset_summary_id_map = generate_summary_id_map(os_summary_info, group_set_summary_info, group_output_set)

outputset_summary_id_map

{9: {1: 2, 2: 1}}

In [16]:
# save outputs
with open(output_dir / 'analysis.json', 'w') as f:
    _analysis_dict = {key: asdict(value) for key, value in analysis.items()}
    json.dump(_analysis_dict, f, indent=4)

with open(output_dir / 'group_output_set.json', 'w') as f:
    json.dump(group_output_set, f, indent=4)

group_set.to_csv(output_dir / 'group_set.csv')
group_event_set_analysis.to_csv(output_dir / 'group_event_set_analysis.csv', index=False)
event_occurrence_set_df.to_csv(output_dir / 'event_occurrence_set.csv', index=False)

outputsets_df.to_csv(output_dir / 'output_set.csv', index=False)

# Serialise summary-info
for gs, g_summary_info_df in group_set_summary_info.items():
    gs_info = group_set.loc[gs]
    summary_info_fname = f'{gs_info['perspective_code']}_GS{gs}_summary-info.csv'
    g_summary_info_df.to_csv(output_dir / summary_info_fname, index=False)

## 2. Period Sampling
Now that each analysis has been grouped, we need to generate the GroupPeriods
into which the events are assigned to for the combined output.

We extract the Period for a given GroupEventSet that has a loss causing event
and the total number of Periods. These Periods are then assigned to the
GroupPeriod randomly, and if the total number of GroupPeriods is larger than
the total number of Period then the GroupEventSet periods are cycled.

In the example below we have a GroupEventSet with 25 total periods and 5 loss
causing events in periods 2, 10, 12, 15 and 21. The total number of GroupPeriods is 100.

In [17]:
from ord_combining.groupperiod import gen_group_periods_event_set_analysis

total_periods = 25
loss_periods = [2, 10, 12, 15, 21]
total_group_periods = 100

example_group_period = gen_group_periods_event_set_analysis(loss_periods,
                                                    max_period=total_periods,
                                                    max_group_periods=total_group_periods)

In [18]:
example_group_period = pd.concat(example_group_period)
example_group_period = example_group_period.sort_values(by='GroupPeriod').reset_index(drop=True)

for i in range(total_group_periods // total_periods):
    slice = (i*total_periods, (i+1) * total_periods)
    print(f"Current cycle: {i+1} : {slice}")
    print(example_group_period[(example_group_period['GroupPeriod'] >= slice[0]) &
                               (example_group_period['GroupPeriod'] < slice[1])])

Current cycle: 1 : (0, 25)
   GroupPeriod  Period
0            3      21
1           15      15
2           16      12
3           18       2
4           19      10
Current cycle: 2 : (25, 50)
   GroupPeriod  Period
5           30      21
6           35       2
7           37      10
8           48      15
9           49      12
Current cycle: 3 : (50, 75)
    GroupPeriod  Period
10           53      10
11           56      12
12           58      15
13           59       2
14           68      21
Current cycle: 4 : (75, 100)
    GroupPeriod  Period
15           76      21
16           79      10
17           88      15
18           94      12
19           98       2


The period information can be extracted from the PLT files or event
occurrence file. In this instance we load the periods from the PLT files.

In [19]:
from ord_combining.groupperiod import generate_group_periods

total_periods = 1000 # config: from model
total_group_periods = 2000 # config: set by user

In [20]:
group_period = generate_group_periods(group_event_set_analysis, analysis, total_periods, total_group_periods)

group_period.head()

,GroupPeriod,Period,group_event_set_id
0,3,177,1
1,6,903,1
2,8,68,1
3,13,833,1
4,15,73,1


In [21]:
# save csv
group_period.to_csv(output_dir / 'group_period.csv', index=False)

**Q**:
- Currently loading Periods from single PLT file in ORD. Should this actually read all PLT files?
- How to load total periods from analysis settings (is it possible to have different total periods for individual grouped analyses)?

## 3. Loss Sampling
The final step involves sampling losses for each event in the GroupPeriod.
There are two types of loss sampling:
- Mean only (only for MELT files)
- Full uncertainty sampling

The additional config options are demonstrated below:

In [22]:
loss_sampling_config = {
        "group_mean" : False,
        "group_mean_type" : 1,  # SampleType filter
        "group_secondary_uncertainty" : False,
        "group_parametric_distribution" : 'gamma', # either gamma or beta
        "group_format_priority": {"M", "Q", "S"}
        }

The first stage in loss sampling is generating the GroupPeriodQuantile table.

In [23]:
from ord_combining.losssampling import construct_gpqt

gpqt = construct_gpqt(group_period, group_event_set_analysis, outputsets_df, analysis,
                      loss_sampling_config.get('group_mean', False))

Currently processing group_event_set_id: 1,  outputset: 0
Currently processing group_event_set_id: 1,  outputset: 1
Currently processing group_event_set_id: 1,  outputset: 2
Currently processing group_event_set_id: 1,  outputset: 3
Currently processing group_event_set_id: 1,  outputset: 4
Currently processing group_event_set_id: 1,  outputset: 5
Currently processing group_event_set_id: 1,  outputset: 6
Currently processing group_event_set_id: 1,  outputset: 7
Currently processing group_event_set_id: 2,  outputset: 8
Currently processing group_event_set_id: 2,  outputset: 9


In [24]:
# save gpqt
gpqt.to_csv(output_dir / "group_period_quantile.csv", index=False)

Finally the loss sampling can be done to produce the group period loss table (GPLT).

In [25]:
from ord_combining.losssampling import do_loss_sampling_full_uncertainty, do_loss_sampling_mean_only

In [26]:
gplt_full = do_loss_sampling_full_uncertainty(gpqt, outputsets_df,
                                              group_output_set, analysis,
                                              priority=['q', 's'],
                                              outputset_summary_id_map=outputset_summary_id_map)
gplt_full.head()

Running output_set_id:0.    1/10


quantile ls: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.28s/it]


Running output_set_id:1.    2/10


quantile ls: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.37s/it]


Running output_set_id:2.    3/10


...

Running output_set_id:8.    9/10


quantile ls: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.40s/it]


Running output_set_id:9.    10/10


quantile ls: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.37s/it]


,group_set_id,output_set_id,SummaryId,GroupPeriod,Period,group_event_set_id,EventId,Loss,LossType
0,0,0,1,3,177,1,261,111667.986244,2
1,0,0,1,6,903,1,1309,15882.957802,2
2,0,0,1,8,68,1,102,694536.764730,2
3,0,0,1,13,833,1,1221,196630.208017,2
4,0,0,1,15,73,1,109,175940.030449,2


In [27]:
gplt_mean = do_loss_sampling_mean_only(gpqt, outputsets_df, group_output_set, analysis,
                                       outputset_summary_id_map=outputset_summary_id_map)

In [28]:
gplt_mean.head()

,group_set_id,output_set_id,SummaryId,GroupPeriod,Period,group_event_set_id,EventId,Loss,LossType
0,0,0,1,3,177,1,261,349520.00,1
1,0,0,1,3,177,1,261,311814.06,3
2,0,0,1,6,903,1,1309,174760.00,1
3,0,0,1,6,903,1,1309,169399.72,3
4,0,0,1,8,68,1,102,349520.00,1


In [29]:
gplt_mean.describe()

,group_set_id,output_set_id,SummaryId,GroupPeriod,Period,group_event_set_id,EventId,Loss,LossType
count,62832.000000,62832.000000,62824.0,62832.000000,62832.000000,62832.000000,62832.000000,6.282400e+04,62824.0
mean,1.616565,4.996244,4.91742,995.724774,417.665330,1.264706,694.332760,1.234713e+05,2.104419
std,1.495093,3.045231,3.055195,573.279799,280.159627,0.441180,420.133061,3.221885e+05,0.994541
min,0.000000,0.000000,1.0,2.000000,1.000000,1.000000,1.000000,1.500000e-01,1.0
25%,1.000000,1.000000,2.0,495.000000,178.000000,1.000000,327.000000,1.222161e+04,1.0
50%,1.000000,6.000000,5.0,1000.000000,376.000000,1.000000,684.000000,3.218316e+04,3.0
75%,1.000000,8.000000,8.0,1486.000000,637.000000,2.000000,1066.000000,8.121200e+04,3.0
max,5.000000,9.000000,10.0,2000.000000,1000.000000,2.000000,1447.000000,3.400000e+06,3.0



## 4. Output Generation
The output options are:
- Group Period Loss Table (GPLT)
  - full (all group_set_id) <-- current implementation
  - file based (each group_set_id in new file) <-- probably better
- Group Average Loss Table (GALT)
- Group Exceedance Probability Table (GEPT)

### GPLT output

In [30]:
sort_cols = ['group_set_id', 'output_set_id', 'SummaryId', 'GroupPeriod']
gplt_full.sort_values(by=sort_cols).to_csv(output_dir / "gplt_full.csv", index=False)
gplt_mean.sort_values(by=sort_cols).to_csv(output_dir / "gplt_mean.csv", index=False)

In [31]:
from ord_combining.grouped_output import generate_al, generate_ep

### GALT Output

In [32]:
aal_full = generate_al(gplt_full, total_group_periods)
aal_mean = generate_al(gplt_mean, total_group_periods)

aal_full.to_csv(output_dir / "aal_full.csv", index=False)
aal_mean.to_csv(output_dir / "aal_mean.csv", index=False)

### GEPT Output

In [33]:
ep_full_df = generate_ep(gplt_full, total_group_periods, oep=True, aep=True)
ep_mean_df = generate_ep(gplt_mean, total_group_periods, oep=True, aep=True)

ep_full_df.to_csv(output_dir / "ep_full.csv", index=False)
ep_mean_df.to_csv(output_dir / "ep_mean.csv", index=False)